In [1]:
from tkinter import *

from aif360.datasets import GermanDataset, AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover, LFR
from aif360.algorithms.postprocessing import RejectOptionClassification, EqOddsPostprocessing, CalibratedEqOddsPostprocessing

from IPython.display import Markdown, display

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import Bias
import Data

RANDOM_STATE=42

In [2]:
def create_semi_german_age():
    def german_preprocessing(df):
        """Adds a derived sex attribute based on personal_status."""
        # TODO: ignores the value of privileged_classes for 'sex'
        status_map = {'A91': 'male', 'A93': 'male', 'A94': 'male',
                      'A92': 'female', 'A95': 'female'}
        df['sex'] = df['personal_status'].replace(status_map)
        
        # ensure 1, 0 mapping instead of 1, 2.
        df["credit"] = df["credit"].apply(lambda x: 1 if x == 1 else 0)
        return df

    german_mappings = {
        'label_maps': [{1.0: 'Good Credit', 0.0: 'Bad Credit'}],
        'protected_attribute_maps': [{1.0: 'Male', 0.0: 'Female'},
                                     {1.0: 'Old', 0.0: 'Young'}],
    }

    # load data
    data = GermanDataset(
        protected_attribute_names=['age'],         
        privileged_classes=[lambda x: x >= 25], # age >=25 is considered privileged
        features_to_drop=['personal_status', 'sex'], # ignore sex-related attributes, as we only focus on age here
        custom_preprocessing=german_preprocessing,
        metadata=german_mappings
    )
    
    # convert to custom object
    dataset_orig = Data.fromAif(data)
    
    # calculate mean difference
    display(Markdown("#### Original training dataset"))
    print(f"Difference in mean outcomes between unprivileged and privileged groups = {dataset_orig.metrics().mean_difference()}")
    
    # fit reweighing
    RW = Reweighing(unprivileged_groups=[dataset_orig.unprivilegedGroups()],
                    privileged_groups=[dataset_orig.privilegedGroups()])
    dataset_transf = Data.fromAif(RW.fit_transform(dataset_orig.aif()))
    
    # test reweighing
    display(Markdown("#### Transformed training dataset"))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % dataset_transf.metrics().mean_difference())
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % dataset_orig.metrics().mean_difference())
    return dataset_transf, dataset_orig

In [3]:
def equal_opt(m):
    a = m.true_positive_rate(privileged=False)
    b = m.true_positive_rate(privileged=True)
    print(a, b)
    return (a - b)

def pred_eq(m):
    c = m.false_positive_rate(privileged=False)
    d = m.false_positive_rate(privileged=True)
    print(c, d)
    return (c - d)

In [4]:
def evaluate(name, d, d2=None, scores=None):
    m = d.metrics()
    scores["mean difference"] = m.mean_difference()
    scores["disparate impact"] = m.disparate_impact()
    scores["consistency"] = m.consistency()[0]
    scores["num neg"] = m.num_negatives()
    scores["num pos"] = m.num_positives()
    if d2 is not None:
        m = d.metrics(d2)
        scores["predictive equality"] = pred_eq(m)
        scores["equal opportunity"] = equal_opt(m)
        
        #scores["equal opportunity"] = m.equal_opportunity_difference()
        #scores["equal odds"] = m.average_abs_odds_difference()


def evall(name, data, method):
    print(name)
    
    scores = dict()
    
    data = data.copy()

    train, test = data.split(0.2)
    train = Data.fromAif(method.fit_transform(train.aif()))
    #try:
    #    test = Data.fromAif(method.transform(test.aif()))
    #except:
    #    test = Data.fromAif(method.fit_transform(test.aif()))

    #model = LogisticRegression(solver='liblinear')
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model = model.fit(train.X(), train.y(), sample_weight=train.weights())
    preds = model.predict(test.X())

    scores["accuracy"] = accuracy_score(test.y(), preds, sample_weight=test.weights())
    
    evaluate(f"{name} (pred)", test.copy().set_labels(preds), test, scores=scores)
    
    return scores
    
def evall_post_proba(name, data, method):
    print(name)
    
    scores = dict()
    
    data = data.copy()

    train, _tmp = data.split(0.4)
    test, val = _tmp.split(0.5)

    # fit model
    #model = LogisticRegression(solver='liblinear')
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model = model.fit(train.X(), train.y(), sample_weight=train.weights())
    proba = model.predict_proba(test.X())
    
    t2 = test.copy()
    t2.set_scores(proba[:, 1])
    
    # fit debiaser
    method.fit(test.aif(), t2.aif())
    
    proba = model.predict_proba(val.X())
    preds = model.predict(val.X())
    
    val_true = val.copy()
    val.set_scores(proba[:, 1])
    val.set_labels(preds)
    
    # predict new results
    new_preds = method.predict(val.aif())
    
    val.set_labels(new_preds.labels)
    
    # evaluate results
    scores["accuracy"] = accuracy_score(val_true.y(), val.y(), sample_weight=val.weights())
    evaluate(f"{name} (pred)", val, val_true, scores)
    
    return scores

def evall_post_pred(name, data, method):
    print(name)
    
    scores = dict()
    
    data = data.copy()

    train, _tmp = data.split(0.4)
    test, val = _tmp.split(0.5)

    # fit model
    #model = LogisticRegression(solver='liblinear')
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model = model.fit(train.X(), train.y(), sample_weight=train.weights())
    preds = model.predict(test.X())
    
    t2 = test.copy()
    t2.set_labels(preds)
    
    # fit debiaser
    method.fit(test.aif(), t2.aif())
    
    preds = model.predict(val.X())
    val_true = val.copy()
    val.set_labels(preds)
    
    # predict new results
    new_preds = method.predict(val.aif())
    
    val.set_labels(new_preds.labels)
    
    # evaluate results
    scores["accuracy"] = accuracy_score(val_true.y(), val.y(), sample_weight=val.weights())
    evaluate(f"{name} (pred)", val, val_true, scores)
    
    return scores

def evall_base(name, data):
    print(name)
    
    scores = dict()
    
    data = data.copy()

    train, test = data.split(0.2)
    
    #model = LogisticRegression(solver='liblinear')
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model = model.fit(train.X(), train.y(), sample_weight=train.weights())
    preds = model.predict(test.X())

    scores["accuracy"] = accuracy_score(test.y(), preds, sample_weight=test.weights())
    evaluate(f"{name}", test.copy().set_labels(preds), test, scores=scores)
    
    return scores
    
def redebias(name, data):
    scores = dict()
    
    display(Markdown(f"# Re-De-Biasing with {name}"))
    scores["base"] = evall_base(f"{name}", data)

    ##### reweighing
    RW = Reweighing(unprivileged_groups=[data.unprivilegedGroups()],
                    privileged_groups=[data.privilegedGroups()])
    scores["reweighing"] = evall(f"reweighing for {name}", data, RW)


    ##### disparate impact remover
    DIR = DisparateImpactRemover(repair_level=1.0,
                                 sensitive_attribute=data._p[0])
    scores["disparate impact remover"] = evall(f"disparate impact remover for {name}", data, DIR)


    ##### LFR
    LFRobj = LFR(unprivileged_groups=[data.unprivilegedGroups()],
                 privileged_groups=[data.privilegedGroups()],
                 k=5,
                 Ax=0.01,
                 Ay=1.0,
                 Az=50.0,
                 print_interval=250,
                 verbose=0,
                 seed=RANDOM_STATE)
    try:
        scores["LFR"] = evall(f"learning fair representations for {name}", data, LFRobj)
    except Exception as e:
        print(e)
        scores["LFR"] = None

    ##### CalibratedEqOddsPostprocessing
    CEOP = CalibratedEqOddsPostprocessing(unprivileged_groups=[data.unprivilegedGroups()],
                                         privileged_groups=[data.privilegedGroups()],
                                         cost_constraint="weighted", # fnr + fpr
                                         seed=RANDOM_STATE)
    scores["Calibrated Equal Odds"] = evall_post_proba(f"Calibrated Equal Odds Postprocessor for {name}", data, CEOP)

    ##### EqOddsPostprocessing
    EOP = EqOddsPostprocessing(unprivileged_groups=[data.unprivilegedGroups()],
                                         privileged_groups=[data.privilegedGroups()],
                                         seed=RANDOM_STATE)
    scores["Equal Odds"] = evall_post_proba(f"Equal Odds Postprocessor for {name}", data, EOP)

    ##### RejectOptionClassification
    ROC = RejectOptionClassification(unprivileged_groups=[data.unprivilegedGroups()],
                                         privileged_groups=[data.privilegedGroups()])
    scores["Reject Option Classification"] = evall_post_pred(f"Reject Option Classification for {name}", data, ROC)
    
    return scores

In [5]:
def evaluate_german_age():
    dataset_transf, dataset_orig = create_semi_german_age()

    scores = dict()
    
    scores["Original"] = redebias("Original", dataset_orig)
    
    scores["Base"] = redebias("Base", dataset_transf)
    
    # Historical Bias
    # less A=0, Y=1
    scores["Historical Bias"] = redebias("Historical Bias", Bias.SamplingBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                           weight={
                                                               "credit": {0:0 , 1:1}
                                                           },
                                                           bias_strength=0.4).apply(dataset_transf))
    
    # Representation Bias
    scores["Representation Bias"] = redebias("Representation Bias", Bias.SamplingBiasGenerator(parameter="age",
                                                              parameter_value=0,
                                                              weight={
                                                                  "skill_level=A174": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="age",
                                                              parameter_value=0,
                                                              weight={
                                                                  "skill_level=A173": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="age",
                                                              parameter_value=0,
                                                              weight={
                                                                  "property=A121": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="age",
                                                              parameter_value=0,
                                                              weight={
                                                                  "housing=A152": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(dataset_transf)
    ))))
    
    # Measurement Bias
    scores["Measurement Bias"] = redebias("Measurement Bias", Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="housing=A153").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="housing=A153").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="savings=A61").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="savings=A63").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="savings=A64").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="savings=A65").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="other_debtors=A101").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="other_debtors=A103").apply(dataset_transf)))))))))
    # Direct Bias
    scores["Direct Bias"] = redebias("Direct Bias", Bias.MeasurementBiasGenerator(parameter="age", 
                                                           parameter_value=0,
                                                          measurement="credit",
                                                           weight={
                                                               "invalid_ratio": 0,
                                                               "measurement_error": [1, 0]
                                                           },
                                                           bias_strength=0.4).apply(dataset_transf))
    
    # Omitted Variable Bias
    scores["Omitted Variable Bias"] = redebias("Omitted Variable Bias", Bias.OmittedVariableBiasGenerator(parameter_to_omit="housing=A151"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="housing=A153"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="savings=A61"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="savings=A63"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="savings=A64"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="savings=A65"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="other_debtors=A101"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="other_debtors=A103"
                                                                       ).apply(dataset_transf)))))))))
    
    return scores

german = evaluate_german_age()

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.12854990969960323


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000
Difference in mean outcomes between unprivileged and privileged groups = -0.128550


# Re-De-Biasing with Original

...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

0.6666666666666666 0.7094594594594594
0.5 0.8125
Calibrated Equal Odds Postprocessor for Omitted Variable Bias
0.14285714285714285 nan
0.7037037037037037 0.7289156626506024
Equal Odds Postprocessor for Omitted Variable Bias



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

0.14285714285714285 0.47619047619047616
0.8333333333333334 0.7597402597402597
Reject Option Classification for Omitted Variable Bias
0.6 0.3888888888888889
0.7619047619047619 0.7564102564102564


Unable to satisy fairness constraints

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
for i in german:
    print(i, end="")
    for j in ["base", "reweighing", "disparate impact remover", "LFR"]:
        if german[i][j] is not None:
            if j != "base":
                print(f" ~ & {j}", end="")
            else:
                print(f" ~ & ~", end="")
                
            for m in german[i][j]:
                if german[i][j][m] is not None:
                    print(f" & {german[i][j][m]:.5}", end="")
            print("\\\\")
    print("\hline\n")

Original ~ & ~ & 0.745 & -0.1237 & 0.85238 & 0.797 & 35.0 & 165.0 & -0.045977 & -0.26\\
 ~ & reweighing & 0.745 & 0.060566 & 1.0811 & 0.756 & 49.0 & 151.0 & 0.54545 & -0.13187\\
 ~ & disparate impact remover & 0.785 & -0.20327 & 0.74253 & 0.716 & 48.0 & 152.0 & -0.027778 & -0.16819\\
 ~ & LFR & 0.285 & 0.0 & nan & 1.0 & 200.0 & 0.0 & -0.032738 & nan\\
\hline

Base ~ & ~ & 0.75 & -0.15 & 0.82857 & 0.816 & 31.0 & 169.0 & -0.17727 & 0.10493\\
 ~ & reweighing & 0.785 & -0.14601 & 0.82817 & 0.79 & 34.0 & 166.0 & -0.019231 & -0.0064447\\
 ~ & disparate impact remover & 0.74 & -0.03248 & 0.96041 & 0.764 & 37.0 & 163.0 & -0.25714 & -0.035654\\
 ~ & LFR & 0.295 & 0.0 & nan & 1.0 & 200.0 & 0.0 & -0.080623 & nan\\
\hline

Historical Bias ~ & ~ & 0.78756 & -0.27641 & 0.65965 & 0.72539 & 44.0 & 149.0 & -0.23325 & -0.095025\\
 ~ & reweighing & 0.80829 & -0.17382 & 0.78241 & 0.75959 & 43.0 & 150.0 & -0.35294 & -0.25926\\
 ~ & disparate impact remover & 0.79275 & -0.35347 & 0.56007 & 0.75233 & 45.0 & 

In [7]:
for i in german:
    print(i, end="")
    for j in ["base", "Calibrated Equal Odds", "Equal Odds", "Reject Option Classification"]:
        if german[i][j] is not None:
            if j != "base":
                print(f" ~ & {j}", end="")
            else:
                print(f" ~ & ~", end="")
                
            for m in german[i][j]:
                if german[i][j][m] is not None:
                    print(f" & {german[i][j][m]:.5}", end="")
            print("\\\\")
    print("\hline\n")

Original ~ & ~ & 0.745 & -0.1237 & 0.85238 & 0.797 & 35.0 & 165.0 & -0.045977 & -0.26\\
 ~ & Calibrated Equal Odds & 0.725 & -0.17647 & 0.82353 & 0.953 & 6.0 & 194.0 & nan & -0.050344\\
 ~ & Equal Odds & 0.755 & -0.19429 & 0.76712 & 0.748 & 38.0 & 162.0 & -0.08046 & 0.024829\\
 ~ & Reject Option Classification & 0.78 & -0.10579 & 0.86305 & 0.723 & 49.0 & 151.0 & 0.047847 & -0.15539\\
\hline

Base ~ & ~ & 0.75 & -0.15 & 0.82857 & 0.816 & 31.0 & 169.0 & -0.17727 & 0.10493\\
 ~ & Calibrated Equal Odds & 0.725 & -0.17647 & 0.82353 & 0.953 & 6.0 & 194.0 & nan & -0.050344\\
 ~ & Equal Odds & 0.755 & -0.19429 & 0.76712 & 0.748 & 38.0 & 162.0 & -0.08046 & 0.024829\\
 ~ & Reject Option Classification & 0.78 & -0.10579 & 0.86305 & 0.723 & 49.0 & 151.0 & 0.047847 & -0.15539\\
\hline

Historical Bias ~ & ~ & 0.78756 & -0.27641 & 0.65965 & 0.72539 & 44.0 & 149.0 & -0.23325 & -0.095025\\
 ~ & Calibrated Equal Odds & 0.73057 & -0.42857 & 0.57143 & 0.93472 & 9.0 & 184.0 & nan & -0.16085\\
 ~ & Equal O

In [8]:
def create_semi_adult_sex():
    # load data
    single_protected = ['sex']
    single_privileged = [['Male']]
    ad = AdultDataset(protected_attribute_names=single_protected,
        privileged_classes=single_privileged,
        features_to_drop=['fnlwgt', "race", "native-country", "relationship", "marital-status"])
    
    # convert to custom object
    dataset_orig = Data.fromAif(ad)
    
    # calculate mean difference
    display(Markdown("#### Original training dataset"))
    print(f"Difference in mean outcomes between unprivileged and privileged groups = {dataset_orig.metrics().mean_difference()}")
    
    # fit reweighing
    RW = Reweighing(unprivileged_groups=[dataset_orig.unprivilegedGroups()],
                    privileged_groups=[dataset_orig.privilegedGroups()])
    dataset_transf = Data.fromAif(RW.fit_transform(dataset_orig.aif()))
    
    
    # test reweighing
    display(Markdown("#### Transformed training dataset"))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % dataset_transf.metrics().mean_difference())
    return dataset_transf, dataset_orig

In [9]:
def evaluate_adult_sex():
    dataset_transf, dataset_orig = create_semi_adult_sex()

    scores = dict()
    
    scores["Original"] = redebias("Original", dataset_orig)
    
    scores["Base"] = redebias("Base", dataset_transf)
    
    # Historical Bias
    # less A=0, Y=1
    scores["Historical Bias"] = redebias("Historical Bias", Bias.SamplingBiasGenerator(parameter="sex", 
                                                           parameter_value=0,
                                                           weight={
                                                               "income-per-year": {0:0 , 1:1}
                                                           },
                                                           bias_strength=0.7).apply(dataset_transf))
    
    # Representation Bias
    scores["Representation Bias"] = redebias("Representation Bias", Bias.SamplingBiasGenerator(parameter="sex",
                                                              parameter_value=0,
                                                              weight={
                                                                  "workclass=Private": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="sex",
                                                              parameter_value=0,
                                                              weight={
                                                                  "occupation=Machine-op-inspct": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="sex",
                                                              parameter_value=0,
                                                              weight={
                                                                  "occupation=Tech-support": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(
                                    Bias.SamplingBiasGenerator(parameter="sex",
                                                              parameter_value=0,
                                                              weight={
                                                                  "occupation=Prof-specialty": {0:0, 1:1}
                                                              },
                                                              bias_strength=0.7).apply(dataset_transf)
    ))))
    
    # Measurement Bias
    scores["Measurement Bias"] = redebias("Measurement Bias", Bias.MeasurementBiasGenerator(parameter="sex", 
                                                           parameter_value=0,
                                                          measurement="hours-per-week").apply(
                                                    Bias.MeasurementBiasGenerator(parameter="sex", 
                                                           parameter_value=0,
                                                          measurement="age").apply(dataset_transf)))
    # Direct Bias
    scores["Direct Bias"] = redebias("Direct Bias", Bias.MeasurementBiasGenerator(parameter="sex", 
                                                           parameter_value=0,
                                                          measurement="income-per-year",
                                                           weight={
                                                               "invalid_ratio": 0,
                                                               "measurement_error": [1, 0]
                                                           },
                                                           bias_strength=0.7).apply(dataset_transf))
    
    # Omitted Variable Bias
    scores["Omitted Variable Bias"] = redebias("Omitted Variable Bias", 
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="hours-per-week"
                                                                       ).apply(
                                      Bias.OmittedVariableBiasGenerator(parameter_to_omit="age"
                                                                       ).apply(dataset_transf)))
    
    return scores

adult = evaluate_adult_sex()

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.19911019753072282


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


# Re-De-Biasing with Original

...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.0826271186440678 0.2099737532808399
0.753968253968254 0.7608024691358025
Reject Option Classification for Omitted Variable Bias



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

0.08735150244584207 0.3092518101367659
0.7769230769230769 nan


invalid value encountered in double_scalars
invalid value encountered in double_scalars


In [10]:
for i in adult:
    print(i, end="")
    for j in ["base", "reweighing", "disparate impact remover", "LFR"]:
        if adult[i][j] is not None:
            if j is not "base":
                print(f" ~ & {j}", end="")
            else:
                print(f" ~ & ~", end="")
                
            for m in adult[i][j]:
                if adult[i][j][m] is not None:
                    print(f" & {adult[i][j][m]:.5}", end="")
            print("\\\\")
    print("\hline\n")

Original ~ & ~ & 0.81872 & -0.20659 & 0.26546 & 0.8694 & 7229.0 & 1978.0 & -0.09779 & -0.081742\\
 ~ & reweighing & 0.81775 & -0.17503 & 0.34363 & 0.8805 & 7279.0 & 1928.0 & -0.11099 & -0.17013\\
 ~ & disparate impact remover & 0.81829 & -0.1874 & 0.29523 & 0.88398 & 7315.0 & 1892.0 & -0.097595 & -0.17055\\
 ~ & LFR & 0.75106 & 0.0 & nan & 1.0 & 9207.0 & 0.0 & -0.20299 & nan\\
\hline

Base ~ & ~ & 0.82068 & -0.19793 & 0.27719 & 0.87334 & 7283.0 & 1924.0 & -0.10268 & -0.12695\\
 ~ & reweighing & 0.81547 & -0.16057 & 0.3608 & 0.87714 & 7376.0 & 1831.0 & -0.11128 & -0.21733\\
 ~ & disparate impact remover & 0.81872 & -0.20574 & 0.28849 & 0.8737 & 7149.0 & 2058.0 & -0.095521 & -0.13172\\
 ~ & LFR & 0.75041 & 0.0 & nan & 1.0 & 9207.0 & 0.0 & -0.20531 & nan\\
\hline

Historical Bias ~ & ~ & 0.83445 & -0.2587 & 0.047917 & 0.8847 & 7248.0 & 1722.0 & -0.15104 & -0.023626\\
 ~ & reweighing & 0.83779 & -0.23133 & 0.11371 & 0.88566 & 7260.0 & 1710.0 & -0.15168 & -0.3246\\
 ~ & disparate impact rem

"is not" with a literal. Did you mean "!="?
"is not" with a literal. Did you mean "!="?
"is not" with a literal. Did you mean "!="?


In [11]:
for i in adult:
    print(i, end="")
    for j in ["base", "Calibrated Equal Odds", "Equal Odds", "Reject Option Classification"]:
        if adult[i][j] is not None:
            if j is not "base":
                print(f" ~ & {j}", end="")
            else:
                print(f" ~ & ~", end="")
                
            for m in adult[i][j]:
                if adult[i][j][m] is not None:
                    print(f" & {adult[i][j][m]:.5}", end="")
            print("\\\\")
    print("\hline\n")

Original ~ & ~ & 0.81872 & -0.20659 & 0.26546 & 0.8694 & 7229.0 & 1978.0 & -0.09779 & -0.081742\\
 ~ & Calibrated Equal Odds & 0.81112 & -0.26846 & 0.0 & 0.87527 & 7535.0 & 1672.0 & -0.061783 & nan\\
 ~ & Equal Odds & 0.81525 & -0.20546 & 0.24385 & 0.88013 & 7313.0 & 1894.0 & -0.11361 & -0.090142\\
 ~ & Reject Option Classification & 0.81807 & -0.19323 & 0.28632 & 0.87781 & 7285.0 & 1922.0 & -0.099538 & -0.14711\\
\hline

Base ~ & ~ & 0.82068 & -0.19793 & 0.27719 & 0.87334 & 7283.0 & 1924.0 & -0.10268 & -0.12695\\
 ~ & Calibrated Equal Odds & 0.81112 & -0.26846 & 0.0 & 0.87527 & 7535.0 & 1672.0 & -0.061783 & nan\\
 ~ & Equal Odds & 0.81525 & -0.20546 & 0.24385 & 0.88013 & 7313.0 & 1894.0 & -0.11361 & -0.090142\\
 ~ & Reject Option Classification & 0.81807 & -0.19323 & 0.28632 & 0.87781 & 7285.0 & 1922.0 & -0.099538 & -0.14711\\
\hline

Historical Bias ~ & ~ & 0.83445 & -0.2587 & 0.047917 & 0.8847 & 7248.0 & 1722.0 & -0.15104 & -0.023626\\
 ~ & Calibrated Equal Odds & 0.83612 & -0.27641

"is not" with a literal. Did you mean "!="?
"is not" with a literal. Did you mean "!="?
"is not" with a literal. Did you mean "!="?


In [17]:
print(adult["Base"]["base"].keys())

dict_keys(['accuracy', 'mean difference', 'disparate impact', 'consistency', 'num neg', 'num pos', 'predictive equality', 'equal opportunity'])
